In [2]:
!pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
import simpy
import random
import scipy.stats as st

In [6]:
SIMULATION_TIME = 1440 # minutes in a day
STATIONS_COUNT = 81
INIT_BIKES_PER_STATION = 10
RIDERS_COUNT = 3500
LAMBDA = 2.38
MEAN_TRIP = 2.78
STD_DEV_TRIP = 0.619

In [7]:
start_station_probs_df = pd.read_csv("start_station_probs.csv")
trip_stats_df = pd.read_csv("trip_stats.csv")
start_station_totals = trip_stats_df.groupby('start')['count'].sum()
trip_stats_df.reset_index()
averages = []
for index, row in trip_stats_df.iterrows():
  s = row['start']
  x = row['count'] / start_station_totals.loc[s]
  averages.append(x)
trip_stats_df['average'] = averages
trip_stats_df = trip_stats_df.drop(columns=['mean', 'std'])
trip_stats_df

,start,end,count,average
0,11 St & Washington St,11 St & Washington St,142,0.078151
1,11 St & Washington St,12 St & Sinatra Dr N,44,0.024216
2,11 St & Washington St,14 St Ferry - 14 St & Shipyard Ln,48,0.026417
3,11 St & Washington St,4 St & Grand St,47,0.025867
4,11 St & Washington St,6 St & Grand St,25,0.013759
...,...,...,...,...
5141,Willow Ave & 12 St,Stevens - River Ter & 6 St,23,0.022352
5142,Willow Ave & 12 St,Vesey Pl & River Terrace,1,0.000972
5143,Willow Ave & 12 St,Warren St,7,0.006803
5144,Willow Ave & 12 St,Washington St,11,0.010690


In [8]:
p_i_j = trip_stats_df['average'].to_numpy()
stations = start_station_probs_df.iloc[:,0].to_numpy()
q_i_j = np.zeros((81,81))
for i in range(q_i_j.shape[0]):
  for j in range(q_i_j.shape[1]):
    try:
      q_i_j[i][j] = trip_stats_df.loc[(trip_stats_df['start'] == stations[i]) & (trip_stats_df['end'] == stations[j])]['average'].iloc[0]
    except:
      q_i_j[i][j] = 0
END_PROBABILITIES = q_i_j.tolist()
STATIONS_PROBABILITIES = start_station_probs_df.iloc[:, 1].to_numpy().tolist()

In [9]:
class Simulation:
  def __init__(self, env):
    self.env = env
    self.total = 0
    self.success_rental = 0
    self.wait_time = 0

  def bike_station(self, env, station):
    yield self.env.timeout(random.expovariate(LAMBDA))
    start_time = self.env.now
    self.total += 1
    station_index = random.choices(range(STATIONS_COUNT), STATIONS_PROBABILITIES)[0]
    while bikes[station_index] == 0:
      yield self.env.timeout(1)
    end_time = self.env.now
    bikes[station_index] -= 1
    self.success_rental += 1
    self.wait_time += (end_time - start_time)
    end_index = random.choices(range(STATIONS_COUNT), END_PROBABILITIES[station_index])[0]
    self.env.process(self.return_bike(station_index, end_index))

  def return_bike(self, station_id, end_index):
    trip_time = random.lognormvariate(MEAN_TRIP, STD_DEV_TRIP)
    yield self.env.timeout(trip_time)
    bikes[end_index] += 1


In [28]:
success_rentals_list = []
average_wait_time_list = []
for i in range(100):
  env = simpy.Environment()
  sim = Simulation(env)

  stations = [simpy.Resource(env) for _ in range(STATIONS_COUNT)]
  bikes = [INIT_BIKES_PER_STATION for _ in range(STATIONS_COUNT)]

  for i in range(RIDERS_COUNT):
    env.process(sim.bike_station(env, i))

  env.run(until=SIMULATION_TIME)
  success_rentals_list.append(sim.success_rental/ sim.total)
  average_wait_time_list.append(sim.wait_time / sim.success_rental) 

In [31]:
success_rentals_df = pd.DataFrame(success_rentals_list)
print(success_rentals_df.mean())
success_rentals_conf_interval = st.t.interval(confidence=0.9, df=len(success_rentals_df)-1, loc=success_rentals_df.mean(), scale=st.sem(success_rentals_df))
print(success_rentals_conf_interval[0][0])
print(success_rentals_conf_interval[1][0])

average_wait_time_df = pd.DataFrame(average_wait_time_list)
average_wait_time_conf_interval = st.t.interval(confidence=0.9, df=len(average_wait_time_df)-1, loc=average_wait_time_df.mean(), scale=st.sem(average_wait_time_df))
print(average_wait_time_conf_interval[0][0])
print(average_wait_time_conf_interval[1][0])
print(average_wait_time_df.mean())

0    0.983014
dtype: float64
0.9818892013022361
0.9841393701263353
34.566191830681085
34.87290634009903
0    34.719549
dtype: float64
